# Анализ эксперимента по подстройке коэффициентов Kp, Ki, Kd с включенной термостабилизацией и другим setpoint от 17 и 19 января 2026 г


In [ ]:
import matplotlib.pyplot as plt

from pathlib import Path
from nn_laser_stabilizer.experiment.config import load_config

EXPERIMENT_DIR = "pid_delta_tuning/2026-01-19_16-49-40"
EXPERIMENT_DIR_PATH = Path(f"../experiments/{EXPERIMENT_DIR}")

ENV_LOG_PATH = EXPERIMENT_DIR_PATH / "env_logs" / "env.log"
TRAIN_LOG_PATH = EXPERIMENT_DIR_PATH / "train_logs" / "train.log"

CONFIG_PATH = EXPERIMENT_DIR_PATH / "config.yaml"
config = load_config(CONFIG_PATH)
print(f"Эксперимент: {config.experiment_name}")

## Анализ логов окружения


In [ ]:
import re
import numpy as np
import pandas as pd

def parse_env_logs(file_path: str | Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    step_pattern = re.compile(
        r"\[ENV\]\s+"
        r"step:\s+"
        r"step=(?P<step>\d+)\s+"
        r"time=(?P<time>-?\d+\.\d+)\s+"
        r"kp=(?P<kp>-?\d+\.\d+)\s+"
        r"ki=(?P<ki>-?\d+\.\d+)\s+"
        r"kd=(?P<kd>-?\d+\.\d+)\s+"
        r"delta_kp_norm=(?P<delta_kp_norm>-?\d+\.\d+)\s+"
        r"delta_ki_norm=(?P<delta_ki_norm>-?\d+\.\d+)\s+"
        r"delta_kd_norm=(?P<delta_kd_norm>-?\d+\.\d+)\s+"
        r"error_mean_norm=(?P<error_mean_norm>-?\d+\.\d+)\s+"
        r"error_std_norm=(?P<error_std_norm>-?\d+\.\d+)\s+"
        r"reward=(?P<reward>-?\d+\.\d+)\s+"
        r"should_reset=(?P<should_reset>\w+)"
    )
    
    reset_pattern = re.compile(
        r"\[ENV\]\s+"
        r"reset:\s+"
        r"time=(?P<time>-?\d+\.\d+)\s+"
        r"kp=(?P<kp>-?\d+\.\d+)\s+"
        r"ki=(?P<ki>-?\d+\.\d+)\s+"
        r"kd=(?P<kd>-?\d+\.\d+)\s+"
        r"error_mean_norm=(?P<error_mean_norm>-?\d+\.\d+)\s+"
        r"error_std_norm=(?P<error_std_norm>-?\d+\.\d+)"
    )
    
    pid_send_pattern = re.compile(
        r"\[PID\]\s+"
        r"send:\s+"
        r"kp=(?P<kp>-?\d+\.\d+)\s+"
        r"ki=(?P<ki>-?\d+\.\d+)\s+"
        r"kd=(?P<kd>-?\d+\.\d+)\s+"
        r"control_min=(?P<control_min>\d+)\s+"
        r"control_max=(?P<control_max>\d+)\s+"
        r"setpoint=(?P<setpoint>\d+)"
    )
    
    pid_read_pattern = re.compile(
        r"\[PID\]\s+"
        r"read:\s+"
        r"process_variable=(?P<process_variable>-?\d+\.\d+)\s+"
        r"control_output=(?P<control_output>-?\d+\.\d+)"
    )
    
    env_rows = []
    conn_rows = []
    current_step = 0
    connection_step = 0
    
    pending_send = None
    line_number = 0  
    
    with open(file_path, 'r') as f:
        for raw_line in f:
            line_number += 1
            line = raw_line.strip()
            if not line:
                continue

            m = pid_send_pattern.match(line)
            if m:
                pending_send = {
                    'kp': float(m.group('kp')),
                    'ki': float(m.group('ki')),
                    'kd': float(m.group('kd')),
                    'control_min': int(m.group('control_min')),
                    'control_max': int(m.group('control_max')),
                    'setpoint': int(m.group('setpoint')),
                    'line': line_number,
                }
                continue
            
            m = pid_read_pattern.match(line)
            if m:
                if pending_send is None:
                    raise ValueError(
                        f"PID READ without matching SEND at line {line_number}: {raw_line!r}"
                    )
                
                conn_rows.append({
                    'Connection step': connection_step,
                    'Type': 'exchange',
                    'Kp': pending_send['kp'],
                    'Ki': pending_send['ki'],
                    'Kd': pending_send['kd'],
                    'Control min': pending_send['control_min'],
                    'Control max': pending_send['control_max'],
                    'Process variable': float(m.group('process_variable')),
                    'Control output': float(m.group('control_output')),
                    'Setpoint': pending_send['setpoint']
                })
                connection_step += 1
                continue
             
            m = reset_pattern.match(line)
            if m:
                env_rows.append({
                    'Step': current_step,
                    'time': float(m.group('time')),
                    'Type': 'reset',
                    'Kp': float(m.group('kp')),
                    'Ki': float(m.group('ki')),
                    'Kd': float(m.group('kd')),
                    'Delta Kp': np.nan,
                    'Delta Ki': np.nan,
                    'Delta Kd': np.nan,
                    'Error mean norm': float(m.group('error_mean_norm')),
                    'Error std norm': float(m.group('error_std_norm')),
                    'Reward': np.nan,
                    'Should reset': True,
                })
                continue
            
            m = step_pattern.match(line)
            if m:
                current_step = int(m.group('step'))
                env_rows.append({
                    'Step': current_step,
                    'time': float(m.group('time')),
                    'Type': 'step',
                    'Kp': float(m.group('kp')),
                    'Ki': float(m.group('ki')),
                    'Kd': float(m.group('kd')),
                    'Delta Kp': float(m.group('delta_kp_norm')),
                    'Delta Ki': float(m.group('delta_ki_norm')),
                    'Delta Kd': float(m.group('delta_kd_norm')),
                    'Error mean norm': float(m.group('error_mean_norm')),
                    'Error std norm': float(m.group('error_std_norm')),
                    'Reward': float(m.group('reward')),
                    'Should reset': m.group('should_reset').lower() == 'true',
                })
                continue
    
    env_df = pd.DataFrame(env_rows)
    connection_df = pd.DataFrame(conn_rows)
    return env_df, connection_df

env_df, connection_df = parse_env_logs(ENV_LOG_PATH)
print(f"Загружено {len(env_df)} записей из логов окружения")
print(f"Загружено {len(connection_df)} записей из логов соединения")


In [ ]:
step_df = env_df[env_df['Type'] == 'step'].copy()
print("=== Статистика по шагам окружения ===")
print(step_df.describe())


In [ ]:
exploration_steps = config.training.exploration_steps
initial_collect_steps = config.training.initial_collect_steps
neural_network_step = max(initial_collect_steps, exploration_steps)

columns_to_plot = ['Error mean norm', 'Error std norm', 'Reward']

for col in columns_to_plot:
    plt.figure(figsize=(12, 5))
    plt.plot(step_df['Step'], step_df[col], alpha=0.8, linewidth=0.8, label=col)
    
    if neural_network_step <= step_df['Step'].max():
        plt.axvline(x=neural_network_step, color='red', linestyle='--', linewidth=2, 
                    label=f'Switch to NN (step {neural_network_step})')
    
    plt.title(f'{col} over Steps')
    plt.xlabel('Step')
    plt.ylabel(col)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

In [ ]:
precision_weight = config.env.args.precision_weight
stability_weight = config.env.args.action_weight
action_weight = config.env.args.stability_weight

step_df['precision_penalty'] = precision_weight * -step_df['Error mean norm'].abs()
step_df['stability_penalty'] = stability_weight * -step_df['Error std norm'].abs()
step_df['action_penalty'] = action_weight * -(
    step_df['Delta Kp'].abs() + 
    step_df['Delta Ki'].abs() + 
    step_df['Delta Kd'].abs()
) / 3.0

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

ax1 = axes[0]
ax1.plot(step_df['Step'], step_df['precision_penalty'], 'b-', alpha=0.7, linewidth=0.8, label='Precision penalty (mean)')
ax1.plot(step_df['Step'], step_df['stability_penalty'], 'r-', alpha=0.7, linewidth=0.8, label='Stability penalty (std)')
ax1.plot(step_df['Step'], step_df['action_penalty'], 'g-', alpha=0.7, linewidth=0.8, label='Action penalty')
ax1.set_ylabel('Penalty Value', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.legend(loc='best')

ax2 = axes[1]
ax2.plot(step_df['Step'], step_df['Reward'], 'm-', alpha=0.8, linewidth=1.0, label='Total Reward')
ax2.set_xlabel('Step', fontsize=12)
ax2.set_ylabel('Reward', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.legend(loc='best')

plt.tight_layout()

In [ ]:
cols = ['Kp', 'Ki', 'Kd']
fig, axes = plt.subplots(len(cols), 1, figsize=(14, 10), sharex=True)

for ax, col in zip(axes, cols):
    ax.plot(step_df['Step'], step_df[col], alpha=0.8, linewidth=0.8, label=col)

    if neural_network_step <= step_df['Step'].max():
        ax.axvline(
            x=neural_network_step,
            color='red',
            linestyle='--',
            linewidth=2,
            label=f'Switch NN ({neural_network_step})'
        )

    ax.set_ylabel(col)
    ax.grid(True, alpha=0.3)
    ax.legend()

axes[-1].set_xlabel("Step")
plt.suptitle("Kp / Ki / Kd over Steps", fontsize=14)
plt.tight_layout()
plt.show()

## Анализ состояния установки


В одном эксперимент есть выброс process variable, который надо отбросить.

In [ ]:
# connection_df = connection_df.drop(connection_df['Process variable'].idxmax())

In [ ]:
block_size = config.env.args.block_size

neural_network_connection_step = neural_network_step * block_size
plt.figure(figsize=(12, 5))
plt.plot(connection_df['Connection step'], connection_df['Process variable'], 'b-', alpha=0.7, linewidth=0.8, label='Process Variable')
plt.plot(connection_df['Connection step'], connection_df['Setpoint'], color='r', linestyle='--')

if neural_network_connection_step <= connection_df['Connection step'].max():
    plt.axvline(x=neural_network_connection_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN (step {neural_network_connection_step})')

plt.title('Process Variable')
plt.xlabel('Step')
plt.ylabel('Process Variable')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(connection_df['Connection step'], connection_df['Control output'], 'g-', alpha=0.7, linewidth=0.8, label='Control Output')

if neural_network_connection_step <= connection_df['Connection step'].max():
    plt.axvline(x=neural_network_connection_step, color='red', linestyle='--', linewidth=2, 
                label=f'Switch to NN (step {neural_network_connection_step})')

plt.title('Control Output')
plt.xlabel('Step')
plt.ylabel('Control Output')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
env_df = env_df.sort_values('time').copy()

env_df['time_diff'] = env_df['time'].diff()
env_df['time_diff_ms'] = env_df['time_diff'] * 1000  

step_time_df = env_df[env_df['Type'] == 'step'].copy()
time_df = env_df.copy()

print("=== Статистика по времени ===")
print(f"Всего записей: {len(time_df)}")
print(f"Шагов: {len(time_df[time_df['Type'] == 'step'])}")
print(f"Reset событий: {len(time_df[time_df['Type'] == 'reset'])}")

if len(step_time_df) > 0:
    print(f"\n=== Статистика интервалов между шагами ===")
    print(step_time_df['time_diff_ms'].describe())
    print(f"\nМедианный интервал: {step_time_df['time_diff_ms'].median():.2f} мс")
    print(f"Средний интервал: {step_time_df['time_diff_ms'].mean():.2f} мс")
    print(f"Максимальный интервал: {step_time_df['time_diff_ms'].max():.2f} мс")
    print(f"Минимальный интервал: {step_time_df['time_diff_ms'].min():.2f} мс")
    
    plt.figure(figsize=(12, 5))
    plt.plot(step_time_df['Step'], step_time_df['time_diff_ms'], 'b-', alpha=0.7, linewidth=0.8)
    plt.title('Time Intervals Between Steps')
    plt.xlabel('Step')
    plt.ylabel('Time Interval (ms)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(12, 5))
    plt.hist(step_time_df['time_diff_ms'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    plt.title('Distribution of Time Intervals Between Steps')
    plt.xlabel('Time Interval (ms)')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## Анализ процесса обучения


In [ ]:
def parse_train_logs(file_path):
    step_pattern = re.compile(
        r"\[TRAIN\]\s+"
        r"step:\s+"
        r"(actor_loss=(?P<actor_loss>-?\d+\.\d+)\s+)?"
        r"buffer_size=(?P<buffer_size>\d+)\s+"
        r"loss_q1=(?P<loss_q1>-?\d+\.\d+)\s+"
        r"loss_q2=(?P<loss_q2>-?\d+\.\d+)\s+"
        r"step=(?P<step>\d+)\s+"
        r"time=(?P<time>-?\d+\.\d+)"
    )
    
    rows = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            match = step_pattern.match(line)
            if match:
                actor_loss = match.group('actor_loss')
                rows.append({
                    'step': int(match.group('step')),
                    'loss_q1': float(match.group('loss_q1')),
                    'loss_q2': float(match.group('loss_q2')),
                    'actor_loss': float(actor_loss) if actor_loss else np.nan,
                    'buffer_size': int(match.group('buffer_size'))
                })
    
    return pd.DataFrame(rows)

loss_df = parse_train_logs(TRAIN_LOG_PATH)
print(f"Загружено {len(loss_df)} записей из логов обучения")
print(f"Диапазон шагов обучения: {loss_df['step'].min()} - {loss_df['step'].max()}")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axes[0].plot(loss_df['step'], loss_df['loss_q1'], 'b-', alpha=0.7, label='Q1 Loss')
axes[0].set_title('Q1 Loss')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(loss_df['step'], loss_df['loss_q2'], 'g-', alpha=0.7, label='Q2 Loss')
axes[1].set_title('Q2 Loss')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(loss_df['step'], loss_df['loss_q1'] + loss_df['loss_q2'], 'r--', alpha=0.7, label='Sum (Q1 + Q2)')
axes[2].set_title('Sum (Q1 + Q2)')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Loss')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
actor_loss_df = loss_df[loss_df['actor_loss'].notna()]
if len(actor_loss_df) > 0:
    plt.plot(actor_loss_df['step'], actor_loss_df['actor_loss'], 'r-', alpha=0.7)
    plt.title('Actor Loss')
else:
    plt.text(0.5, 0.5, 'No actor loss data', ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('Actor Loss (no data)')
plt.xlabel('Step')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(loss_df['step'], loss_df['buffer_size'], 'm-', alpha=0.7)
plt.title('Buffer Size')
plt.xlabel('Step')
plt.ylabel('Size')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()